In [1]:
import requests
import folium
import pandas as pd
from sklearn.cluster import KMeans
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import json
import requests
from bs4 import BeautifulSoup as bSoup
import numpy as np
import re
from geopy.geocoders import Nominatim as geocoder
import csv
from pandas import json_normalize

print("Library import")
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = bSoup(data,"html5lib")

contents = []
table = soup.find('table')
for row in table.findAll('td'):
    col = {}
    #Skip cells of table with not assinged.
    if row.span.text == 'Not assigned':
        pass
    #Split the contents of table cell, where first 3 chars are the postal code, and borough is contained outside the (),
    #and the neighbourhood is within the ().
    else:
        col['PCode'] = row.p.text[:3]
        col['Borough'] = (row.span.text).split('(')[0]
        col['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(col)
tblCont = pd.DataFrame(contents)
tblCont['Borough'] = tblCont['Borough'].replace({'EtobicokeNorthwest':'Etobicoke Northwest',
                                            'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                                            'East YorkEast Toronto':'East York/East Toronto'})
print("Successful scrap")


Library import
Successful scrap


#### Csv file from the url contains the long and lat of Toronto, just have to parse out the ones that match the scraped postal codes.


In [2]:
longLatCSV ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
parsePCodes = pd.read_csv(longLatCSV)

#### Join the parsePCodes with tblCont on postal codes, and remove added columns from the dataframe that have NaN data, delete duplicate data, and reset the index.

In [3]:
finalizedLatLong = pd.merge(tblCont, parsePCodes, left_on = 'PCode', right_on = 'Postal Code')
finalizedLatLong = finalizedLatLong.drop('Postal Code', axis = 1)
finalizedLatLong = finalizedLatLong.drop_duplicates()
finalizedLatLong.index = range(len(finalizedLatLong))
finalizedLatLong.head()

,PCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


#### Foursquare api info

In [4]:
lim = 50
ver = '20220409'
c_id = 'TPIL203UWTGQRQCXIVERR4XCXJOYE0PC5GRVKYAFGA2JWPQD'
c_srct_id = 'M5LN5VA54524SARZLVUCE0VXEZHHSNQ2AURODZBW3PEQJHLL'
couProvCity = 'Toronto, Ontario, Canada'

#### Use foursquare agent

In [5]:
gLoc = geocoder(user_agent = "AI-Proj-Key-2022")
cityLoc = gLoc.geocode(couProvCity)
latitude = cityLoc.latitude
longitude = cityLoc.longitude
tlongitude = longitude
tlatitude = latitude

mapOfRegion = folium.Map(location = [latitude, longitude], zoom_start = 11)
#mapOfRegion

#### Place markers on neighbourhoods that have been scraped.

In [6]:
for lati, longi, borough, neighbourhood in zip(finalizedLatLong['Latitude'], finalizedLatLong['Longitude'],finalizedLatLong['Borough'],finalizedLatLong['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lati, longi],
    fill = True,
    fill_opacity = 0.1,
    color = '#cc8131',
    fill_color = 'yellow',
    radius = 6,
    popup = label,
    parse_html = False).add_to(mapOfRegion)
mapOfRegion

#### Reading the needed csv files to datasets to join on common columns.

In [7]:
dataCrime = pd.read_csv ('Neighbourhood_Crime_Rates_(Boundary_File)_.csv')
dataNeighProfiles = pd.read_csv ('neighbourhood-profiles-2016-csv.csv')
dataNeigh = pd.read_csv ('Neighbourhoods-4326.csv')
#print(dataNeighProfiles)

#### Joining dataNeigh with dataCrime on the same Neighbourhood name and AREA_DESC name.

In [8]:
crimeAndNeigh = dataCrime.merge(dataNeigh,left_on = 'Neighbourhood', right_on = 'AREA_DESC')
#print(crimeAndNeigh)

#### fgdf

In [9]:
totalDF = pd.merge(crimeAndNeigh, dataNeighProfiles, on = 'Neighbourhood')
totalDF.to_csv('totalDF.csv')